# Recipe Diversity: Variant A/B/C Screening
**Goal**: Train 3 recipe variants x 3 seeds each = 9 models for screening.
Mix with existing 23 base seeds for greedy ensemble selection.

**Variants** (all h=64, 3L, vanilla+linear, raw32, no norm):
- **Base** (gru_parity_v1): LR=0.001, WD=0, drop=0, MSE, ReduceOnPlateau
- **Variant A**: LR=0.0005, WD=1e-5, drop=0.05 (mild regularization)
- **Variant B**: LR=0.002, CosineAnnealing (different schedule)
- **Variant C**: Huber loss delta=1.0 (different error shape)

**Screen criteria**: variant contributes 2+ models to greedy top-10 -> scale up to 8 seeds.

In [ ]:
# Cell 0: Mount Drive, download data from Kaggle
import os, json

from google.colab import drive
drive.mount('/content/drive')
os.makedirs('/content/drive/MyDrive/wunderfund', exist_ok=True)

!pip install -q kaggle==1.6.14 --force-reinstall
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump({"username": "vincentvdo6", "key": "FILL_IN"}, f)
os.chmod('/root/.kaggle/kaggle.json', 0o600)

os.makedirs('/content/data', exist_ok=True)
!kaggle datasets download -d vincentvdo6/wunderfund-predictorium -p /content/data/ --force
!unzip -o -q /content/data/wunderfund-predictorium.zip -d /content/data/
!ls /content/data/*.parquet

In [ ]:
# Cell 1: Setup -- clone repo, link data
import os, subprocess
REPO = "/content/competition_package"

os.chdir("/content")
subprocess.run(["rm", "-rf", REPO], check=False)
subprocess.run(["git", "clone", "https://github.com/vincentvdo6/competition_package.git", REPO], check=True)
os.chdir(REPO)
os.makedirs("datasets", exist_ok=True)
os.makedirs("logs", exist_ok=True)

subprocess.run(["ln", "-sf", "/content/data/train.parquet", "datasets/train.parquet"], check=True)
subprocess.run(["ln", "-sf", "/content/data/valid.parquet", "datasets/valid.parquet"], check=True)

assert os.path.exists("datasets/train.parquet"), "train.parquet not found!"
assert os.path.exists("datasets/valid.parquet"), "valid.parquet not found!"
commit = subprocess.check_output(["git", "rev-parse", "--short", "HEAD"], text=True).strip()
print(f"Commit: {commit}")
print(f"GPU: {subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], text=True).strip()}")
print("Ready!")

In [ ]:
# Cell 2: Train recipe variants (3 seeds each for screening)
import os, subprocess, sys
os.chdir("/content/competition_package")

VARIANTS = [
    ("configs/vanilla_varA.yaml", "varA", [42, 43, 44]),
    ("configs/vanilla_varB.yaml", "varB", [42, 43, 44]),
    ("configs/vanilla_varC.yaml", "varC", [42, 43, 44]),
]

print("=== RECIPE DIVERSITY TRAINING ===")
print(f"Variants: {len(VARIANTS)}")
for cfg, name, seeds in VARIANTS:
    print(f"  {name}: {cfg} x seeds {seeds}")
print("=" * 60, flush=True)

for config_path, variant_name, seeds in VARIANTS:
    for seed in seeds:
        print(f"\n{'='*60}")
        print(f"Training {variant_name} seed {seed}")
        print(f"Config: {config_path}")
        print(f"{'='*60}", flush=True)
        proc = subprocess.Popen(
            [sys.executable, "-u", "scripts/train.py",
             "--config", config_path,
             "--seed", str(seed), "--device", "cuda"],
            stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
        )
        for line in proc.stdout:
            print(line, end="", flush=True)
        proc.wait()
        if proc.returncode != 0:
            print(f"ERROR: {variant_name} seed {seed} failed with rc={proc.returncode}")

print(f"\nAll training done!")

In [ ]:
# Cell 3: Evaluate all variants vs base recipe
import os, glob, torch
from collections import defaultdict
os.chdir("/content/competition_package")

BASE_MEAN_VAL = 0.2689  # mean of 23 base seeds

results = []
for pattern in ["logs/vanilla_var*_seed*.pt", "logs/gru_parity_v1_seed*.pt"]:
    for pt in sorted(glob.glob(pattern)):
        basename = os.path.basename(pt)
        if '_epoch' in basename:
            continue
        ckpt = torch.load(pt, map_location="cpu", weights_only=False)
        score = float(ckpt.get("best_score", 0))
        epoch = ckpt.get("best_epoch", "N/A")
        name = basename.replace(".pt", "")
        results.append((name, score, epoch))

results.sort(key=lambda x: -x[1])

# Group by variant
by_variant = defaultdict(list)
for name, score, epoch in results:
    if "parity_v1" in name:
        variant = "base"
    elif "varA" in name:
        variant = "varA"
    elif "varB" in name:
        variant = "varB"
    elif "varC" in name:
        variant = "varC"
    else:
        variant = "other"
    by_variant[variant].append(score)

print("=== RECIPE COMPARISON ===")
print(f"{'Variant':<10} {'N':>3} {'Mean':>8} {'Best':>8} {'Worst':>8} {'vs Base':>8}")
print("-" * 55)
for variant in ["base", "varA", "varB", "varC"]:
    scores = by_variant.get(variant, [])
    if not scores:
        print(f"{variant:<10} {'--':>3}")
        continue
    mean_s = sum(scores) / len(scores)
    delta = mean_s - BASE_MEAN_VAL
    print(f"{variant:<10} {len(scores):>3} {mean_s:>8.4f} {max(scores):>8.4f} "
          f"{min(scores):>8.4f} {delta:>+8.4f}")

print(f"\nAll seeds ranked:")
print(f"{'Rank':<6} {'Name':<40} {'Val Score':>10} {'Epoch':>6}")
print("-" * 65)
for i, (name, score, epoch) in enumerate(results[:30], 1):
    print(f"{i:<6} {name:<40} {score:>10.4f} {str(epoch):>6}")

In [ ]:
# Cell 4: Strip checkpoints + zip + save to Drive
import os, torch, glob, shutil
os.chdir("/content/competition_package")
os.makedirs("logs/slim", exist_ok=True)

for pt in sorted(glob.glob("logs/vanilla_var*_seed*.pt")):
    basename = os.path.basename(pt)
    if '_epoch' in basename:
        continue
    ckpt = torch.load(pt, map_location="cpu", weights_only=False)
    slim = {
        "model_state_dict": ckpt["model_state_dict"],
        "config": ckpt.get("config", {}),
        "best_score": ckpt.get("best_score", None),
        "best_epoch": ckpt.get("best_epoch", None),
    }
    out = f"logs/slim/{basename}"
    torch.save(slim, out)
    orig = os.path.getsize(pt) / 1e6
    new = os.path.getsize(out) / 1e6
    print(f"{basename}: {orig:.1f}MB -> {new:.1f}MB")

shutil.make_archive("/content/recipe_diversity_seeds", "zip",
                     "/content/competition_package/logs/slim")
sz = os.path.getsize("/content/recipe_diversity_seeds.zip") / 1e6
print(f"\nrecipe_diversity_seeds.zip: {sz:.1f}MB")

shutil.copy("/content/recipe_diversity_seeds.zip",
            "/content/drive/MyDrive/wunderfund/recipe_diversity_seeds.zip")
print("Saved to Drive: MyDrive/wunderfund/recipe_diversity_seeds.zip")

In [ ]:
# Cell 5: Scale up winning variant (uncomment appropriate block after Cell 3)
import os, subprocess, sys
os.chdir("/content/competition_package")

# === SET THE WINNER AFTER EVALUATING CELL 3 ===
# WINNER_CONFIG = "configs/vanilla_varA.yaml"
# WINNER_CONFIG = "configs/vanilla_varB.yaml"
# WINNER_CONFIG = "configs/vanilla_varC.yaml"
WINNER_CONFIG = None  # <-- Set this!

if WINNER_CONFIG:
    SCALE_SEEDS = list(range(45, 53))  # 8 more seeds
    print(f"Scaling up: {WINNER_CONFIG} x {len(SCALE_SEEDS)} seeds")
    print(f"Seeds: {SCALE_SEEDS}")
    print("=" * 60, flush=True)
    for seed in SCALE_SEEDS:
        print(f"\n{'='*60}")
        print(f"Training seed {seed}")
        print(f"{'='*60}", flush=True)
        proc = subprocess.Popen(
            [sys.executable, "-u", "scripts/train.py",
             "--config", WINNER_CONFIG,
             "--seed", str(seed), "--device", "cuda"],
            stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
        )
        for line in proc.stdout:
            print(line, end="", flush=True)
        proc.wait()
        if proc.returncode != 0:
            print(f"ERROR: seed {seed} failed")
    print("\nScale-up done!")
    
    # Strip and save scale-up checkpoints
    import glob, torch, shutil
    variant = os.path.basename(WINNER_CONFIG).replace('.yaml', '')
    os.makedirs("logs/slim", exist_ok=True)
    for pt in sorted(glob.glob(f"logs/{variant}_seed*.pt")):
        basename = os.path.basename(pt)
        if '_epoch' in basename:
            continue
        ckpt = torch.load(pt, map_location="cpu", weights_only=False)
        slim = {
            "model_state_dict": ckpt["model_state_dict"],
            "config": ckpt.get("config", {}),
            "best_score": ckpt.get("best_score", None),
            "best_epoch": ckpt.get("best_epoch", None),
        }
        out = f"logs/slim/{basename}"
        torch.save(slim, out)
        print(f"{basename}: stripped")
    
    shutil.make_archive(f"/content/{variant}_scaleup", "zip",
                         "/content/competition_package/logs/slim")
    sz = os.path.getsize(f"/content/{variant}_scaleup.zip") / 1e6
    print(f"\n{variant}_scaleup.zip: {sz:.1f}MB")
    shutil.copy(f"/content/{variant}_scaleup.zip",
                f"/content/drive/MyDrive/wunderfund/{variant}_scaleup.zip")
    print("Saved to Drive!")
else:
    print("Set WINNER_CONFIG first based on Cell 3 results!")